# YOLOv12n Zero-shot Finetuning and Baseline

## Imports

In [1]:
import os, sys

# if colab, mount drive and get the git repo
if 'google.colab' in sys.modules:
    from google.colab import drive
    print(os.getcwd())
    drive.mount('/content/drive')
    !git clone --recurse-submodules https://github.com/jsluijter02/LightSB_YOLO

    # Append LightSB_YOLO path
    sys.path.append(os.path.join(os.getcwd(), 'LightSB_YOLO'))

    !unzip -q /content/drive/MyDrive/data.zip -d /content/LightSB_YOLO/

# otherwise local path append
else:
    sys.path.append(os.path.dirname(os.getcwd()))

In [2]:
# append the yolo model to path
from scripts.utils import dirs
dirs.add_YOLOPX_to_PATH()
from scripts.models.yolo import YOLOPX_BDD

from ultralytics import YOLO
from scripts.dataset.ultralytics_format import export_to_ultralytics_format, yaml_file
from scripts.dataset import bdd

from scripts.utils.device import get_device

/opt/anaconda3/envs/lightsb_yolo/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/opt/anaconda3/envs/lightsb_yolo/lib/python3.8/site-packages/albumentations/__init__.py:13: UserWarning: A new version of Albumentations is available: 2.0.8 (you have 1.4.18). Upgrade using: pip install -U albumentations. To disable automatic update checks, set the environment variable NO_ALBUMENTATIONS_UPDATE to 1.
  check_for_updates()


## Data

### Get DB's

In [ ]:
img_dir = os.path.join(dirs.get_data_dir(), 'bdd', 'images')
px = YOLOPX_BDD()
train_day = bdd.get_db(px.config, is_train=True, timeofday='daytime')
val_day = bdd.get_db(px.config, is_train=False, timeofday='daytime')

AUTO_RESUME: False
CUDNN:
  BENCHMARK: True
  DETERMINISTIC: False
  ENABLED: True
DATASET:
  CLAHE_CLIPLIMIT: 2.0
  CLAHE_VAL: False
  COLOR_RGB: False
  DATAROOT: /Users/jochem/Documents/GitHub/LightSB_YOLO/data/images
  DATASET: BddDataset
  DATA_FORMAT: jpg
  FLIP: True
  HSV_H: 0.015
  HSV_S: 0.7
  HSV_V: 0.4
  LABELROOT: /Users/jochem/Documents/GitHub/LightSB_YOLO/data/bdd/det_annotations
  LANEROOT: /Users/jochem/Documents/GitHub/LightSB_YOLO/data/bdd/ll_seg_annotations
  MASKROOT: /Users/jochem/Documents/GitHub/LightSB_YOLO/data/bdd/da_seg_annotations
  ORG_IMG_SIZE: [720, 1280]
  ROT_FACTOR: 10
  SCALE_FACTOR: 0.25
  SELECT_DATA: False
  SHEAR: 0.0
  TEST_SET: val
  TRAIN_SET: train
  TRANSLATE: 0.1
DEBUG: False
GPUS: (0,)
LOG_DIR: runs/
LOSS:
  BOX_GAIN: 0.05
  CLS_GAIN: 0.5
  CLS_POS_WEIGHT: 1.0
  DA_SEG_GAIN: 0.2
  FL_GAMMA: 2.0
  LL_IOU_GAIN: 0.2
  LL_SEG_GAIN: 0.2
  LOSS_NAME: 
  MULTI_HEAD_LAMBDA: None
  OBJ_GAIN: 1.0
  OBJ_POS_WEIGHT: 1.0
  SEG_POS_WEIGHT: 1.0
MODEL:
  

=> creating runs/BddDataset/_2026-02-04-14-40


/Users/jochem/Documents/GitHub/LightSB_YOLO/scripts/models/yolo.py:75: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.load(self.weights, map_location=self.

### Make Ultralytics files

In [4]:
label_dir = os.path.join(dirs.get_data_dir(), 'labels', 'train')
export_to_ultralytics_format(train_day, save_dir=label_dir)
label_dir = os.path.join(dirs.get_data_dir(), 'labels', 'val')
export_to_ultralytics_format(val_day, save_dir=label_dir)

yaml_path = os.path.join(dirs.get_data_dir(), 'bdd.yaml')
yaml_file(train_day, train_img_path=os.path.join(dirs.get_data_dir(), 'images', 'train'), val_img_path=os.path.join(dirs.get_data_dir(), 'images', 'val'), yaml_path=yaml_path)

 50%|█████     | 18580/36800 [00:09<00:09, 1916.31it/s]


KeyboardInterrupt: 

## Model

In [ ]:
model = YOLO('yolo26n.pt')
print(model.info())

YOLO26n summary: 260 layers, 2,572,280 parameters, 0 gradients, 6.1 GFLOPs
(260, 2572280, 0, 6.1192448)


In [ ]:
results = model.train(
    data="/Users/jochem/Documents/GitHub/LightSB_YOLO/data/bdd.yaml",
    epochs=1,
    imgsz=640,
    batch=32,
    fraction=0.05,
    device="mps",
    workers=8,
    mosaic=0.0,
    mixup=0.0,
    copy_paste=0.0,
    plots=False,
    val=False
)
print(results)

Ultralytics 8.4.11 🚀 Python-3.8.20 torch-2.4.1 MPS (Apple M4 Pro)
engine/trainer: agnostic_nms=False, amp=True, angle=1.0, augment=False, auto_augment=randaugment, batch=32, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=/Users/jochem/Documents/GitHub/LightSB_YOLO/data/bdd.yaml, degrees=0.0, deterministic=True, device=mps, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, end2end=None, epochs=1, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=0.05, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolo26n.pt, momentum=0.937, mosaic=0.0, multi_scale=0.0, name=train11, nbs=64, nms=False, opset=None, optimize=False, optimizer=auto, overlap_mask=True, patience=100,

RuntimeError: shape mismatch: value tensor of shape [31243] cannot be broadcast to indexing result of shape [19302]

In [ ]:
!conda list

# packages in environment at /opt/anaconda3/envs/lightsb_yolo:
#
# Name                    Version                   Build  Channel
accelerate                1.0.1                    pypi_0    pypi
albucore                  0.0.17                   pypi_0    pypi
albumentations            1.4.18                   pypi_0    pypi
annotated-types           0.7.0                    pypi_0    pypi
anyio                     4.5.2                    pypi_0    pypi
appnope                   0.1.4              pyhd8ed1ab_0    conda-forge
asttokens                 3.0.0              pyhd8ed1ab_0    conda-forge
backcall                  0.2.0              pyh9f0ad1d_0    conda-forge
ca-certificates           2026.1.4             hbd8a1cb_0    conda-forge
certifi                   2025.11.12               pypi_0    pypi
charset-normalizer        3.4.4                    pypi_0    pypi
click                     8.1.8                    pypi_0    pypi
comm                      0.2.2              pyh